# 22. 딥러닝 레이어의 이해(1) Linear, Convolution

## 1. 들어가며

### 학습 목표
---
1. 레이어의 개념을 이해한다.
2. 딥러닝 모델 속 각 레이어(Linear, Convolution)의 동작 방식을 이해한다.
3. 데이터의 특성을 고려한 레이어를 설계하고, 이를 Tensorflow로 정의하는 법을 배운다.

딥러닝은  $y = Wx + b$에서 최적의 $W$(Weight)과 $b$를 찾는 과정!

이라는 문장을 한번은 들어본 적 있을 것이다. 하지만 막장 이를 누군가에게 설명하려고 하면 쉽지 않다. 우리가 정의하는 수많은 신경망들은 각기 다른 형태의 Weight를 갖고 있고, 그마다 독특한 특성을 가지고 있다. 데이터에서 원하는 특징을 효과적으로 추출하기 위해선 __바른 Weight를 정의하는 과정__이 중요하다. 그리고 그 과정은 하나의 직관이 되어 신경망을 분석하거나 설계하는 데에 큰 도움이 될 것이다.

오늘은 데이터의 차원 변화를 좇으며 각기 다른 신경망들이 갖는 Weight의 특성을 살펴보고, 앞서 언급한 미싱 링크의 진정한 의미를 두 번에 걸쳐 알아볼 것이다. `Linear` 레이어와 `Convolution` 레이어를 집중적으로 공부하고, 다음번에는 `Embedding` 레이어와 `Recurrent` 레이어를 다루어 볼 것이다.

## 2. 데이터의 형태
딥러닝을 이해하는 방법 중 가장 쉬운 방법은 데이터의 형태 변화를 쫓는 것이다. 10개 단어의 문장을 __1)__ 5개 단어로 요약했다면 그것은 정보를 집약시킨 경우일 것이고, __2)__ 20개 단어로 확장했다면 그것은 정보를 더 세밀하게 표현한 경우일 것이다.

아래는 기업 A에 지원한 지원자들의 인적사항이다.

<img src="./image/table1.png" alt="table" />

<표 1>의 데이터는 (3, 6)의 매트릭스로 표현이 가능하다. 만약 기업 B의 데이터가 추가된다면 어떨까?

<img src="./image/table2.png" alt="table" />

<표 2>의 데이터는 (2, 3, 6)의 매트릭스로 표현이 가능하다. 그럼 이미지 데이터는 어떨까?

이미지 데이터는 보통 채널(Channel)이라는 것을 가진다. 우리가 일상적으로 사용하는 것은 대부분 RGB 이미지인데, 여기서 RGB는 Red 채널, Green 채널, 그리고 Blue 채널까지 총 3개의 채널을 의미한다.

<img src="./image/red.png" alt="red" width="30%" />

위 이미지가 바로 3개의 채널 중 Red 채널만을 나타낸 이미지이다. 같은 맥락으로 Green 채널과 Blue 채널도 추출할 수 있고, 모든 채널을 한 데 겹치면 눈에 익숙한 RGB 이미지가 되는 것이다.

<img src="./image/combine.png" alt="combine" width="50%" />

<img src="./image/pixel.png" width="70%" />

RGB 이미지라고 가정을 한다면, 위 그림의 데이터는 `(3, 1920, 1080)` 또는 `(1920, 1080, 3)` 의 매트릭스로 표현이 가능하다. 그리고 표현 방식에 따라 __C__hannel, __W__idth, __H__eight의 이니셜로 `(C, W, H)`, `(W, H, C)`와 같이 표기한다. 이 표기법을 알고 있다면, 어떤 딥러닝 모듈이 요구하는 데이터 형태를 알아보고 그에 맞게 데이터를 변형시킬 수 있다.

## 3. 레이어는 어렵다?
__레이어(layer)__ 라는 단어는 제법 많은 곳에서 쓰이고 있다. 통신분야에서는 OSI 7 레이어를 언급하고, 포토샵에서도 레이어가 쓰이며, 심지어 레이어드 패션이 유행한 적도 있었다. 약간씩 다른 의미로 사용되고 있는 용어지만, 이해해야 할 개념에 가장 가까운 정의는 다음과 같다.

> _하나의 물체가 여러 개의 논리적인 객체들로 구성되어 있는 경우, 이러한 각각의 객체를 하나의 레이어라 한다._

신경망이라는 물체를 구성하는 여러 개의 논리적인 레이어들을 이해하는 것은 곧 신경망 그 자체를 이해하는 것과 같은 것이다. 서론의 Weight 이야기를 다시 생각해보자. 신경망의 Weight라고 표현했지만, 정확히는 __레이어의 Weight__가 맞다. 신경망은 레이어들의 각기 다른 Weight, 그것들이 유기적으로 연결되어 이뤄내는 하나의 결과물인 것이다. 그것은 마치 인간의 두뇌와 닮아서 __인공 신경망__이라는 이름을 갖게 된다.

<img src="./image/layer.png" alt="layers" width="60%" />

이제 각각의 레이어가 어떤 특징을 가지며, 어떻게 쓰이면 좋은지 알아보자.

## 4. 딥러닝의 근본! Linear 레이어
Fully Connected Layer, Feedforward Neural Network, Multilayer Perceptrons, Dense Layer… 등 다양한 이름으로 불리지만 그 모든 것들은 결국 Linear 레이어에 해당하기 때문에 Linear 레이어라고 칭하겠다. 선형대수학에서 쓰이는 용어 중 선형 변환(Linear Transform)이 있는데, 그것과 완전히 동일한 기능을 하는 레이어이다. 아래 영상을 참고하면 도움이 될 것이다.

* [Linear transformations and matrices | Essence of linear algebra, chapter 3](https://www.youtube.com/watch?v=kYB8IZa5AuE&feature=youtu.be&ab_channel=3Blue1Brown)

* [행렬과 선형변환(feat.마인크래프트 스티브) Linear Transformation](https://www.youtube.com/watch?v=vVvjYzFBUVk&feature=youtu.be&ab_channel=%EC%83%81%EC%9A%B0%EC%8C%A4%EC%9D%98%EC%88%98%ED%95%99%EB%85%B8%ED%8A%B8)

Linear 레이어는 선형 변환을 활용해 데이터를 특정 차원으로 변환하는 기능을 한다. 100차원의 데이터를 300차원으로 변환한다면 데이터를 더 풍부하게 표현하는 효과가 있고, 반대로 10차원의 데이터로 변환한다면 데이터를 집약시키는 효과가 있다. 예를 들어보자.

<img src="./image/class.png" alt="class" width="80%" />

위 그림의 두 사각형은 모두 (x, y) 2차원의 점 4개로 표현 가능하므로, 각각 (4, 2) 형태의 데이터로 표현할 수 있다. 두 사각형을 각각 어떤 하나의 정수로 표현하고자 한다. 실은 이 정수는 우리가 구분하고자 하는 사각형의 종류(class)를 의미한다. 이를 위해, 데이터를 어떻게 집약시킬지 구상해보자.

>__<식1>__
<br>1단계: __*(4, 2) x [2 x 1 행렬] = (4, )*__
<br>2단계: __*(4, ) x [4 x 1 행렬] = (1, )*__

위 단계를 사용하면 각각의 사각형을, 정보가 집약된 하나의 정수로 표현할 수 있다. 2차원을 1차원으로 변환하는 데에 2 x 1 행렬이 하나 선언되고, 4차원을 1차원으로 변환하는 데에 4 x 1 행렬이 하나 선언됨에 유의하자. 여기서 각각의 행렬들이 __Weight__이다. Linear 레이어는 `(입력의 차원, 출력의 차원)`에 해당하는 Weight를 가지는 특성을 가지고 있다.

이 과정을 코드로 표현하면 아래와 같다.

In [1]:
import tensorflow as tf

batch_size = 64
boxes = tf.zeros((batch_size, 4, 2))     # Tensorflow는 Batch를 기반으로 동작하기에,
                                         # 우리는 사각형 2개 세트를 batch_size개만큼
                                         # 만든 후 처리를 하게 됩니다.
print("1단계 연산 준비:", boxes.shape)

first_linear = tf.keras.layers.Dense(units=1, use_bias=False) 
# units은 출력 차원 수를 의미합니다.
# Weight 행렬 속 실수를 인간의 뇌 속 하나의 뉴런 '유닛' 취급을 하는 거죠!

first_out = first_linear(boxes)
first_out = tf.squeeze(first_out, axis=-1) # (4, 1)을 (4,)로 변환해줍니다.
                                           # (불필요한 차원 축소)

print("1단계 연산 결과:", first_out.shape)
print("1단계 Linear Layer의 Weight 형태:", first_linear.weights[0].shape)

print("\n2단계 연산 준비:", first_out.shape)

second_linear = tf.keras.layers.Dense(units=1, use_bias=False)
second_out = second_linear(first_out)
second_out = tf.squeeze(second_out, axis=-1)

print("2단계 연산 결과:", second_out.shape)
print("2단계 Linear Layer의 Weight 형태:", second_linear.weights[0].shape)

1단계 연산 준비: (64, 4, 2)
1단계 연산 결과: (64, 4)
1단계 Linear Layer의 Weight 형태: (2, 1)

2단계 연산 준비: (64, 4)
2단계 연산 결과: (64,)
2단계 Linear Layer의 Weight 형태: (4, 1)


<img src="./image/linear1.png" alt="linear" width="80%" />

하지만 순탄치 않다. 위 그림을 보니 두 사각형에 대해 1단계를 거치고 난 결과가 동일한 모습이다. 이렇게 되면 <식 1>의 2단계 입력이 동일해지니 두 번째 4 x 1 Weight를 거치는 것이 의미가 없어진다. 여기서 모든 Weight의 모든 요소를 __Parameter__라고 한다. 총 6개 (위 그림에서는 2개)의 Parameter로 이 문제를 해결하기엔 역부족이었던 것 같다. 첫 번째 접근은 데이터를 집약하는 데에만 집중했으니, 이번엔 데이터를 더 풍부하게 만들어보자.

>__<식2>__
<br>1단계: __*(4, 2) x [2 x 3 행렬] = (4, 3)*__
<br>2단계: __*(4, 3) x [3 x 1 행렬] = (4, )*__
<br>2단계: __*(4, ) x [4 x 1 행렬] = (1, )*__

<img src="./image/linear2.png" alt="linear" width="80%" />

1단계의 결과로 각 사각형에 대해 독립적인 정보가 생겨나기 시작한다. 예컨대 <식 2>는 첫 번째 접근에 비해 더 많은 사각형을 구분해낼 수 있을 것 같다.

<식 1>의 과정을 코드로 작성했던 것처럼, 이번에는 <식 2>의 과정을 Tensorflow 코드로 작성해보자.<br>
작성하며 데이터 차원의 변화와 각 Weight의 형태를 확인하고, `tf.keras.layers.Layer.count_params()` 함수를 사용해서 총 parameter 개수를 계산해보자.

In [2]:
import tensorflow as tf

batch_size = 64
boxes = tf.zeros((batch_size, 4, 2))

print("1단계 연산 준비:", boxes.shape)

first_linear = tf.keras.layers.Dense(units=3, use_bias=False)
first_out = first_linear(boxes)

print("1단계 연산 결과:", first_out.shape)
print("1단계 Linear Layer의 Weight 형태:", first_linear.weights[0].shape)

print("\n2단계 연산 준비:", first_out.shape)

second_linear = tf.keras.layers.Dense(units=1, use_bias=False)
second_out = second_linear(first_out)
second_out = tf.squeeze(second_out, axis=-1)

print("2단계 연산 결과:", second_out.shape)
print("2단계 Linear Layer의 Weight 형태:", second_linear.weights[0].shape)

print("\n3단계 연산 준비:", second_out.shape)

third_linear = tf.keras.layers.Dense(units=1, use_bias=False)
third_out = third_linear(second_out)
third_out = tf.squeeze(third_out, axis=-1)

print("3단계 연산 결과:", third_out.shape)
print("3단계 Linear Layer의 Weight 형태:", third_linear.weights[0].shape)

total_params = \
first_linear.count_params() + \
second_linear.count_params() + \
third_linear.count_params()

print("총 Parameters:", total_params)

1단계 연산 준비: (64, 4, 2)
1단계 연산 결과: (64, 4, 3)
1단계 Linear Layer의 Weight 형태: (2, 3)

2단계 연산 준비: (64, 4, 3)
2단계 연산 결과: (64, 4)
2단계 Linear Layer의 Weight 형태: (3, 1)

3단계 연산 준비: (64, 4)
3단계 연산 결과: (64,)
3단계 Linear Layer의 Weight 형태: (4, 1)
총 Parameters: 13


"그렇다면 Parameter가 많은 것이 최고인가요?" 라고 물을 수 있다. 정답은 그렇지 않다. 지나치게 많은 Parameter는 과적합(Overfitting)을 야기한다. 과적합은 쉽게 말하면 학생이 문제만 보고 정답을 외우는 것과 같다. 정답만 외운 학생은 실제 시험에서 좋은 성적을 거둘 수 없을 것이다.

이런 방법들로 Weight의 형태만 선언해주면 그 파라미터 값을 임의의 실수가 채우고, 수많은 데이터를 거치며 가장 적합한 Weight를 알아서 찾아가는 과정이 바로 훈련(Training)이다. 적합한 파라미터라는 것은 주어진 데이터가 가지는 분포에 따라 결정된다: 뉴스 데이터를 학습한 인공지능이 소설을 잘 쓰리라 기대할 수는 없다. 따라서 다양한 데이터가 많으면 많을수록 실제 세계에 가까운 인공지능이 만들어지게 된다.

추가로 이야기할 것은 바로 편향(Bias)이다. 웃어른을 만났을 때에 우리는 배꼽 인사를 하며 "안녕하세요~" 라고 하겠지만 프랑스에서 배꼽 인사와 "봉주르~"를 하는 것은 상상이 가지 않는다… 이때 인사라는 행위를 $y = wx$라 한다면 우리나라의 인사법을 $y = wx+유교사상$, 프랑스의 인사법을 $y = wx+아메리카 마인드$라고 표현해 보면 어떨까? 각 문화에 편향되었음을 보여주는 것이다. 아래 그림을 보자.

<img src="./image/bias.png" alt="bias" width="80%" />

두 데이터가 비슷하게 생겼지만, 원점을 건들지 않고 둘을 일치시키기는 어려워 보인다. 편향이 없다면 파라미터를 아무리 돌리고 늘리고 해도 정확하게 근사할 수 없음을 보여주는 예이다. 단순히 생각해서 원점을 평행이동하는 것만으로도 해결할 수 있기 때문에 실제로 편향은 선형변환된 값에 편향 파라미터 b를 더해주는 것으로 표현한다. 서론의 $y=Wx+b$속의 b가 바로 그 편향 값이다.

$Wx$ 에 단순히 더하기 때문에 편향 값은 형태가 [ 선형변환 결과 차원, ] 인 한 줄짜리 Weight로 정의된다. 앞서 공부한 예제에서 Dense 클래스 속 `use_bias` 파라미터를 True 로 바꿔주면 실험해볼 수 있다. 이 또한 Weight의 파라미터와 동일하게 수많은 데이터를 통해 적합한 값을 찾아가게 된다. 

## 5. 정보를 집약시키자! Convolution 레이어
Linear 레이어를 배우고 나니 웬만한 데이터는 다 다룰 수 있을 것 같다. 형태에 맞는 Weight만 선언해주면 되기 때문이다 하지만 다음과 같은 데이터를 만난다면 어떨까?

>__<식3>__
<br>1단계: __*(1920, 1080, 3) → (1920 x 1080 x 3, )*__
<br>2단계: __*(6220800, ) x [6220800 x 1 Weight] = (1, )*__

<식 3>에 따르면 아무리 적어도 620만 개의 Parameter가 생성된다. 게다가 "손"이라는 목적이 있음에도 모든 픽셀을 한 줄씩 살펴야하기 때문에 비효율적이다. 그래서 고안된 것이 __Convolution 레이어__이다.

Convolution 연산이라는 것은 딥러닝 외적으로도 많이 사용되는 개념이다. 아래 그림을 통해 필터가 이미지와 겹쳐지는 부분의 Convolution 연산을 통해 새롭게 얻어지는 변환된 이미지가 어떻게 생성되는지 확인해보자.

<img src="./image/conv.png" alt="Convolution" width="80%" />

In [18]:
import os

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [21]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

Physical devices cannot be modified after being initialized


In [23]:
import tensorflow as tf

batch_size = 64
pic = tf.zeros((batch_size, 1920, 1080, 3))

print("입력 이미지 데이터:", pic.shape)
conv_layer = tf.keras.layers.Conv2D(filters=16,
                                    kernel_size=(5, 5),
                                    strides=5,
                                    use_bias=False)
conv_out = conv_layer(pic)

print("\nConvolution 결과:", conv_out.shape)
print("Convolution Layer의 Parameter 수:", conv_layer.count_params())

flatten_out = tf.keras.layers.Flatten()(conv_out)
print("\n1차원으로 펼친 데이터:", flatten_out.shape)

linear_layer = tf.keras.layers.Dense(units=1, use_bias=False)
linear_out = linear_layer(flatten_out)

print("\nLinear 결과:", linear_out.shape)
print("Linear Layer의 Parameter 수:", linear_layer.count_params())

ResourceExhaustedError: OOM when allocating tensor with shape[64,1920,1080,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Fill]